In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'
ab_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\repo\\base'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


CONFIG

In [98]:
# Budget config file
include_pheno = False
threshold_year = 1970
data_path = data_dir
ab_path = ab_dir
max_length = [51,37]
mask_prob = 0.15
embedding_dim = 32
drop_prob = 0.2

#Encoder
enc_dim_inp = 32 
enc_dim_out = 32 
attention_heads = 8 

#BERT
num_encoders = 2

#trainer
epochs = 5
batch_size = 32
lr = 0.001
stop_patience = 10


Data import

In [99]:
from data_preprocessing import data_loader
include_pheno = False
threshold_year = 1970


NCBI, ab_df = data_loader(include_pheno,threshold_year,data_dir,ab_dir)
NCBI.head()

,year,location,genes
1,[PAD],USA,"[sul1, tet(A), aadA1]"
4,[PAD],USA,"[aph(3'')-Ib, tet(A), aph(6)-Id, sul2]"
5,[PAD],USA,"[pmrB_Y358N=POINT, glpT_E448K=POINT]"
6,[PAD],Sweden,"[cyaA_S352T=POINT, glpT_E448K=POINT, uhpT_E350..."
7,1979,USA,[glpT_E448K=POINT]


Vocabulary

In [100]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
include_pheno = False
vocabulary = vocab_geno(NCBI, include_pheno)
vocab = vocab_pheno(ab_df)
print(len(vocabulary))
print(len(vocab))


1208
81


In [92]:
sequences

[['meropenem=R'],
 ['meropenem=R'],
 ['meropenem=S'],
 ['tigecycline=S',
  'nitrofurantoin=S',
  'amikacin=S',
  'levofloxacin=R',
  'ampicillin-sulbactam=R',
  'meropenem=S',
  'amoxicillin-clavulanic acid=S',
  'ampicillin-sulbactam=S',
  'aztreonam=R',
  'trimethoprim-sulfamethoxazole=R',
  'ertapenem=S',
  'amoxicillin-clavulanic acid=R',
  'tobramycin=S',
  'piperacillin-tazobactam=S',
  'ceftazidime=R',
  'gentamicin=S',
  'cefepime=R',
  'tetracycline=R',
  'moxifloxacin=R',
  'ampicillin=R',
  'imipenem=S',
  'ciprofloxacin=R',
  'cefazolin=R',
  'ceftriaxone=R'],
 ['tigecycline=S',
  'nitrofurantoin=S',
  'meropenem=R',
  'amikacin=S',
  'cefotaxime=R',
  'levofloxacin=R',
  'ampicillin-sulbactam=R',
  'cefuroxime=R',
  'aztreonam=R',
  'trimethoprim-sulfamethoxazole=R',
  'ertapenem=S',
  'amoxicillin-clavulanic acid=R',
  'tobramycin=S',
  'ceftazidime=R',
  'gentamicin=S',
  'tetracycline=R',
  'moxifloxacin=R',
  'ampicillin=R',
  'imipenem=S',
  'ciprofloxacin=R',
  'pipe

In [151]:
i = 60
print(len(list_idx[i]))
print(len(list_SR[i]))

37
37


In [149]:
max_length = [51,37]
sequences = deepcopy(NCBI['AST_phenotypes'].tolist())
list_idx = []
list_SR = []
for i in range(len(sequences)):
    current_seq = sequences[i]
    current_idxs = []
    current_SRs = []
    for j in range(len(current_seq)):
        item = current_seq[j].split('=')
        abs = item[0]   
        sr = item[1]
        current_idxs.append(vocab_pheno_test.lookup_indices([abs]))
        for k in range(len(sr)):
            if sr == 'R':
                current_SRs.append(1)
            else:
                current_SRs.append(0)
    current_idxs = [int(item[0]) for item in current_idxs]
    for i in range(0,max_length[1] - len(current_idxs)):
        current_idxs.append(-1)
    for i in range(0,max_length[1] - len(current_SRs)):
        current_SRs.append(-1)
    list_idx.append(current_idxs)
    list_SR.append(current_SRs)

for i in range(len(list_idx)):
    print(list_idx[i])
    print(list_SR[i])


[49, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[49, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[49, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[75, 57, 0, 46, 4, 49, 2, 4, 6, 78, 38, 2, 76, 61, 19, 42, 12, 72, 53, 3, 43, 30, 11, 25, -1, -1, -1, -1, -1,

In [80]:
max_length = [88,51,37]
sequences = deepcopy(NCBI['AST_phenotypes'].tolist())
list_idx = []
list_SR = []
for i in range(len(sequences[5])):
    item = sequences[5][i].split('=')
    abs = item[0]   
    sr = item[1]
    print(item)
    list_idx.append(vocab_pheno_test.lookup_indices([abs]))
    for j in range(len(sr)):
        if sr == 'R':
            list_SR.append(1)
        else:
            list_SR.append(0)
list_idx = [int(item[0]) for item in list_idx]
print(list_idx)
print(list_SR)

['gentamicin', 'S']
['ceftiofur', 'S']
['chloramphenicol', 'R']
['trimethoprim-sulfamethoxazole', 'S']
['cefoxitin', 'S']
['azithromycin', 'S']
['tetracycline', 'R']
['ciprofloxacin', 'S']
['sulfisoxazole', 'R']
['ceftriaxone', 'S']
['nalidixic acid', 'S']
['streptomycin', 'R']
['ampicillin', 'R']
['amoxicillin-clavulanic acid', 'S']
['kanamycin', 'S']
[42, 22, 29, 78, 16, 5, 72, 30, 69, 25, 54, 66, 3, 2, 45]
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]


In [118]:
import pandas as pd
import numpy as np
from pathlib import Path
import torch
from copy import deepcopy
from torch.utils.data import Dataset
from torchtext.vocab import vocab


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MASK_PERCENTAGE = 0.15


# data, vocabulary, max sequence length, mask probability, include sequences, some random state
class NCBIDataset(Dataset):

    MASKED_INDICES_COLUMN = 'masked_indices'
    TARGET_COLUMN = 'indices'
    TOKEN_MASK_COLUMN = 'token_mask'
    AB_INDEX = 'ab_index'
    SR_CLASS = 'sr_class'

    def __init__(self,
                 data: pd.DataFrame,
                 vocab_geno: vocab,
                 vocab_pheno: vocab,
                 max_seq_len: list,
                 mask_prob: float,
                 include_pheno:bool,
                 random_state: int = 23,
                 ):
        
        self.random_state = random_state
        np.random.seed(self.random_state)

        CLS = '[CLS]'
        PAD = '[PAD]'
        MASK = '[MASK]'
        UNK = '[UNK]'

        self.include_pheno = include_pheno
        self.data = data.reset_index(drop=True) 
        self.num_samples = self.data.shape[0]
        self.vocab_geno = vocab_geno
        self.vocab_pheno = vocab_pheno
        self.vocab_size_geno = len(self.vocab_geno)
        self.CLS = CLS 
        self.PAD = PAD
        self.MASK = MASK
        self.UNK = UNK
        self.max_seq_len = max_seq_len
        self.mask_prob = mask_prob
        if self.pheno:
            self.columns = [self.MASKED_INDICES_COLUMN, self.TARGET_COLUMN, self.AB_INDEX, self.SR_CLASS]
        else:
            self.columns = [self.MASKED_INDICES_COLUMN, self.TARGET_COLUMN]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        input = torch.Tensor(item[self.MASKED_INDICES_COLUMN],device=device).long()
        token_mask  = torch.tensor(item[self.TARGET_COLUMN], device=device).long()
        attention_mask = (input == self.vocab_geno[self.PAD]).unsqueeze(0)
        
        if self.include_pheno:
            ab_idx  = torch.tensor(item[self.AB_INDEX], device=device).long()
            sr_class = torch.tensor(item[self.SR_CLASS], device=device).long()
            return input, token_mask , attention_mask, ab_idx, sr_class
        else:
            return input, token_mask , attention_mask

    def _construct_masking(self):
        sequences = deepcopy(self.data['genes'].tolist())
        masked_sequences = []
        target_indices_list = []
        seq_starts = [[self.CLS, self.data['year'].iloc[i], self.data['location'].iloc[i]] for i in range(self.data.shape[0])]

        for i, geno_seq in enumerate(sequences):
            seq_len = len(geno_seq)
            masking_index = np.random.rand(seq_len) < self.mask_prob   
            target_indices = np.array([-1]*seq_len)
            indices = masking_index.nonzero()[0]
            target_indices[indices] = self.vocab_geno.lookup_indices([geno_seq[i] for i in indices])
            for i in indices:
                r = np.random.rand()
                if r < 0.8:
                    geno_seq[i] = self.MASK
                elif r > 0.9:
                    geno_seq[i] = self.vocab_geno.lookup_token(np.random.randint(self.vocab_size_geno))
            geno_seq = seq_starts[i] + geno_seq
            target_indices = [-1]*3 + target_indices.tolist() 
            masked_sequences.append(geno_seq)
            target_indices_list.append(target_indices)
        masked_sequences = [seq + [self.PAD]*(self.max_seq_len[0] - len(seq)) for seq in masked_sequences]
        for i in range(len(target_indices_list)):
            indices = target_indices_list[i]
            padding = [-1] * (self.max_seq_len[0] - len(indices))
            target_indices_list[i] = indices + padding
        return masked_sequences, target_indices_list 
    
    def _Ab_SR_indexing(self):
        sequences = deepcopy(self.data['AST_phenotypes'].tolist())
        list_idx = []
        list_SR = []
        for i in range(len(sequences)):
            current_seq = sequences[i]
            current_idxs = []
            current_SRs = []
            for j in range(len(current_seq)):
                item = current_seq[j].split('=')
                abs = item[0]   
                sr = item[1]
                current_idxs.append(self.vocab_pheno.lookup_indices([abs]))
                for k in range(len(sr)):
                    if sr == 'R':
                        current_SRs.append(1)
                    else:
                        current_SRs.append(0)
            current_idxs = [int(item[0]) for item in current_idxs]
            for i in range(0,max_length[1] - len(current_idxs)):
                current_idxs.append(-1)
            for i in range(0,max_length[1] - len(current_SRs)):
                current_SRs.append(-1)
            list_idx.append(current_idxs)
            list_SR.append(current_SRs)
        return list_idx, list_SR
    
    def prepare_dataset(self):
        masked_sequences, target_indices = self._construct_masking()
        indices_masked = [self.vocab_geno.lookup_indices(masked_seq) for masked_seq in masked_sequences]
        if self.include_pheno:
            list_idx, list_SR = self._Ab_SR_indexing()

        if self.include_pheno:
            rows = zip(indices_masked, target_indices, list_idx, list_SR)
            self.df = pd.DataFrame(rows, columns=self.columns)
        else:
            rows = zip(indices_masked, target_indices)
            self.df = pd.DataFrame(rows, columns=self.columns)

In [119]:
from build_vocabulary import vocab
from build_vocabulary import vocab_pheno
from data_preprocessing import data_loader

include_pheno = True
threshold_year = 1970

data_path = data_dir
ab_path = ab_dir

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)

max_length = [51,37]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

(tensor([  0,   4,  18,  59,  49, 220,  57,  73,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1]),
 tensor([-1, -1, -1, -1, -1, 66, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
 tensor([[False, False, False, False, False, False, False, False,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True]]))

In [ ]:
from create_dataset import NCBIDataset
from build_vocabulary import make_vocabulary
from data_preprocessing import data_loader

include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)

max_length = 51
mask_prob = 0.50
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

Embedding 

In [64]:
from embedding import JointEmbedding
embedding_dim = 32
voca_size = len(vocabulary)
max_length = [88,51,37]
drop_prob = 0.2

emb_test = JointEmbedding(embedding_dim, voca_size, max_length, drop_prob)
emb_test.forward(test_set[1][0])
print(emb_test)

JointEmbedding(
  (token_emb): Embedding(1208, 32)
  (dropout): Dropout(p=0.2, inplace=False)
  (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
)


In [ ]:
from bert_builder import AttentionHead

attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
from bert_builder import MultiHeadAttention
multihead_test = MultiHeadAttention(num_heads=8, dim_inp=32, dim_out=32, drop_prob=0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
from bert_builder import Encoder

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
from bert_builder import BERT

bert_test = BERT(vocab_size=len(vocabulary), max_length=51, dim_inp=32, dim_out=32, attention_heads=8, num_encoders=2, dropout_prob=0.2)
bert_test.forward(test_set[1][0], test_set[1][2]).size()

---------------------------

Trainer

In [ ]:
import numpy as np
from create_dataset import NCBIDataset
def get_split_indices(size_to_split, val_share, random_state: int = 42):
    indices = np.arange(size_to_split)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    train_share = 1 - val_share
    
    train_size = int(train_share * size_to_split)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    return train_indices, val_indices
max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

In [83]:
import time
from torch import nn
from torch.utils.data import DataLoader
import wandb
from pathlib import Path

class BertTrainer:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, wandb_mode, project_name, wandb_name):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.01
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        
        self.wandb_mode = wandb_mode
        self.project_name = project_name
        self.wandb_name = wandb_name
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)

        self.device = device


    def __call__(self):   
        if self.wandb_mode:
            self._init_wandb()   
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss = self.train(self.current_epoch)
            self.train_losses.append(avg_epoch_loss) 
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses.append(val_results[0])  
            self.val_accs.append(val_results[1])
            if self.wandb_mode:
                self._report_epoch_results()
            self._report_epoch_results()
            criterion = self.stop_early()
            if criterion:
                print(f"Training interrupted at epoch: {self.current_epoch+1}")
                break

        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.current_epoch,
            "train_losses": self.train_losses,
            "val_losses": self.val_losses,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):
        self.train_losses = []
        self.val_losses = []
        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss = 0
        reporting_loss = 0
        printing_loss = 0
        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask = batch
            
            self.optimizer.zero_grad() 

            tokens = self.model(input, attn_mask) 
            loss = self.criterion(tokens.transpose(-1, -2), token_target) 
            
            epoch_loss += loss.item() 
            reporting_loss += loss.item()
            printing_loss += loss.item()
            
            loss.backward() 
            self.optimizer.step()         
        avg_epoch_loss = epoch_loss / self.num_batches
        return avg_epoch_loss 
    
    def evaluate(self, loader):
        self.model.eval()
        epoch_loss = 0
        total_correct = 0
        total_tokens = 0

        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask = batch
                tokens = self.model(input, attn_mask)
                loss = self.criterion(tokens.transpose(-1, -2), token_target)
                epoch_loss += loss.item()
                
                token_mask =  token_target != -1
                predicted_tokens = tokens.argmax(dim=-1)
                token_target = torch.masked_select(token_target, token_mask)
                predicted_tokens = torch.masked_select(predicted_tokens, token_mask)
                
                correct = (predicted_tokens == token_target).sum().item()
                total_correct += correct
                total_tokens += token_target.numel() 
        
        avg_epoch_loss = epoch_loss / len(loader)
        accuracy = total_correct / total_tokens

        return avg_epoch_loss, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    def _init_wandb(self):
        self.wandb_run = wandb.init(
            project=self.project_name, # name of the project
            name=self.wandb_name, # name of the run
            
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "num_heads": self.model.attention_heads,
                "num_encoders": self.model.num_encoders,
                "emb_dim": self.model.dim_inp,
                'ff_dim': self.model.dim_out,
                "lr": self.lr,
                "weight_decay": self.weight_decay,
                "max_seq_len": self.model.max_length[0],
                "vocab_size": len(self.train_set.vocab),
                "num_parameters": sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            }
        )
        self.wandb_run.watch(self.model) # watch the model for gradients and parameters
        self.wandb_run.define_metric("epoch", hidden=True)
        self.wandb_run.define_metric("batch", hidden=True)

        self.wandb_run.define_metric("Losses/train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("Losses/val_loss", summary="min", step_metric="epoch")
        
        self.wandb_run.define_metric("Losses/final_val_loss")
        self.wandb_run.define_metric("Accuracies/final_val_acc")
        self.wandb_run.define_metric("final_epoch")

        return self.wandb_run
    
    def _report_epoch_results(self):
        wandb_dict = {
            "epoch": self.current_epoch+1,
            "Losses/train_loss": self.train_losses[-1],
            "Losses/val_loss": self.val_losses[-1],
            "Accuracies/val_acc": self.val_accs[-1],
        }
        self.wandb_run.log(wandb_dict)

In [84]:
from bert_builder import BERT
from data_preprocessing import data_loader
from build_vocabulary import make_vocabulary
from misc import get_split_indices
from create_dataset import NCBIDataset

include_pheno = False

NCBI = data_loader(include_pheno,threshold_year,path)
vocabulary = make_vocabulary(NCBI, include_pheno)
reduced_samples = 1000
NCBI = NCBI.head(reduced_samples)
print(f"Data loaded, {len(NCBI)} samples found")
print(f"length  of vocabulary:",len(vocabulary))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob,include_pheno)

bert_test = BERT(len(vocabulary), max_length, enc_dim_inp, enc_dim_out, attention_heads, num_encoders, drop_prob)

test = BertTrainer(bert_test, train_set, val_set, epochs, batch_size, lr, device, stop_patience, True, "NCBI", "test")
test()

Data loaded, 1000 samples found
length  of vocabulary: 1208


Epoch 1/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:06
Epoch 2/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:12
Epoch 3/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:18
Epoch 4/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:24
Epoch 5/5
Epoch completed in 0.1 min
Evaluating on validation set...
Elapsed time: 00:00:31
-=Training completed=-


{'best_epoch': 4,
 'train_losses': [6.716255130767823,
  5.805294799804687,
  5.029448509216309,
  4.463122625350952,
  4.1071098518371585],
 'val_losses': [6.11516353062221,
  5.36531925201416,
  4.712193965911865,
  4.294982944216047,
  4.067011458533151],
 'val_accs': [0.24260355029585798,
  0.23668639053254437,
  0.1893491124260355,
  0.1952662721893491,
  0.2485207100591716]}

In [ ]:
test._load_model('c:\\Users\\erika\\Desktop\\Exjobb\\test_model.pth')

------------------------------

In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()